In [1]:
import os
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.optim as optim

from hmmlearn import hmm

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convolutional Autoencoder Structure:

In [3]:
# Define the Convolutional Autoencoder (CAE) architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()

        self.ls = 32

        # Define separate encoders for each signal
        self.ecg_encoder = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=20, stride=10, padding=1),
            nn.ReLU(),
            nn.Conv1d(8, self.ls, kernel_size=10, stride=5, padding=1),
        )
        self.rsp_encoder = self.ecg_encoder
        self.eda_tonic_encoder = self.ecg_encoder
        self.eda_phasic_encoder = self.ecg_encoder

        # Fully connected layer to compress the latent space
        self.fc = nn.Linear(128, self.ls)

        # Fully connected layer to decompress the latent space
        self.fc_decoded = nn.Linear(self.ls, 128)

        # Define separate decoders for each signal
        self.ecg_decoder = nn.Sequential(
            nn.ConvTranspose1d(self.ls, 8, kernel_size=10, stride=5, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(8, 1, kernel_size=20, stride=10, padding=1, output_padding=2),
            nn.Sigmoid(),
        )
        self.rsp_decoder = self.ecg_decoder
        self.eda_tonic_decoder = self.ecg_decoder
        self.eda_phasic_decoder = self.ecg_decoder

    def encode(self, x):
        # Split input by channel for independent processing
        ecg = x[:, 0, :].unsqueeze(1)
        rsp = x[:, 1, :].unsqueeze(1)
        eda_tonic = x[:, 2, :].unsqueeze(1)
        eda_phasic = x[:, 3, :].unsqueeze(1)

        # Encode each signal independently
        ecg_encoded = self.ecg_encoder(ecg)
        rsp_encoded = self.rsp_encoder(rsp)
        eda_tonic_encoded = self.eda_tonic_encoder(eda_tonic)
        eda_phasic_encoded = self.eda_phasic_encoder(eda_phasic)

        # Concatenate the latent representations along the last dimension
        latent_space = torch.cat(
            (ecg_encoded, rsp_encoded, eda_tonic_encoded, eda_phasic_encoded), dim=1
        )

        latent_space = latent_space.permute(0, 2, 1)

        # Compress the latent space
        latent_space = self.fc(latent_space)

        return latent_space

    def decode(self, latent_space):
        # Decompress the latent space
        latent_space = self.fc_decoded(latent_space)
        latent_space = latent_space.permute(0, 2, 1)

        # Split latent space back into separate channels
        ecg_latent, rsp_latent, eda_tonic_latent, eda_phasic_latent = torch.split(
            latent_space, self.ls, dim=1
        )
        # Decode each signal independently
        ecg_decoded = self.ecg_decoder(ecg_latent)
        rsp_decoded = self.rsp_decoder(rsp_latent)
        eda_tonic_decoded = self.eda_tonic_decoder(eda_tonic_latent)
        eda_phasic_decoded = self.eda_phasic_decoder(eda_phasic_latent)

        # Concatenate the decoded signals to form the output
        reconstructed = torch.cat(
            (ecg_decoded, rsp_decoded, eda_tonic_decoded, eda_phasic_decoded), dim=1
        )

        return reconstructed

    def forward(self, x):
        latent_space = self.encode(x)
        reconstructed = self.decode(latent_space)
        return reconstructed

In [4]:
learning_rate = 0.001

# Train Autoencoders to Each Participants' Baseline Data

In [5]:
segment_size = "1s"
step_size = "0.001s"

In [6]:
# loop through baseline data
for file in os.listdir("./Physiological Preprocessed/Exp2"):
    participant = file.split("_")[0]
    if "baseline" not in file:
        continue
    elif os.path.exists(f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth"):
        print(f"Model for {file} already exists")
        continue

    print(f"Loading {participant} data")
    print(f"-" * 50)

    # load data
    physiological_data = pd.read_csv(f"./Physiological Preprocessed/Exp2/{file}", usecols=["Time", "ECG_Clean", "RSP_Clean", "EDA_Tonic", "EDA_Phasic"])
    physiological_data["Time"] = pd.to_timedelta(physiological_data["Time"])
    physiological_data.set_index("Time", inplace=True)

    print(f"Processing participant {participant} data")

    # Normalize the data
    scalar = MinMaxScaler()
    data = scalar.fit_transform(physiological_data)
    physiological_data = pd.DataFrame(data, columns=physiological_data.columns, index=physiological_data.index)

    # Split the data into sliding windows
    X = []
    len_segment = pd.Timedelta(segment_size) / pd.Timedelta(step_size)
    while len(physiological_data) > 0:
        start_index = physiological_data.index[0]
        end_index = start_index + pd.Timedelta(segment_size)
        segment = physiological_data[:end_index]
        physiological_data = physiological_data[end_index:]

        if len(segment) > len_segment:
            length = len(segment) - len_segment
            segment = segment.drop(segment.tail(int(length)).index)

        if len(segment) == len_segment:
            X.append(segment.to_numpy())

    X = np.stack(X)

    # Convert to PyTorch tensors
    X = torch.tensor(X, dtype=torch.float32).to(device)

    # Initialize the model
    model = ConvAutoencoder().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

    # Training parameters
    batch_size = 4
    num_epochs = 300

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for i in range(0, len(X), batch_size):

            # Get the batch and reshape it for Conv1d (batch, channels, sequence_length)
            batch = X[i : i + batch_size].permute(0, 2, 1)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch)

            # Calculate loss
            loss = criterion(outputs, batch)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(X):.4f}")

    # Save the model
    if not os.path.exists("./Convolutional Autoencoder Models/Takeover Time"):
        os.makedirs("./Convolutional Autoencoder Models/Takeover Time")
    torch.save(model.state_dict(), f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth")

Model for ST44_baseline.csv already exists
Model for ST16_baseline.csv already exists
Model for NST85_baseline.csv already exists
Model for ST56_baseline.csv already exists
Model for ST04_baseline.csv already exists
Model for NST19_baseline.csv already exists
Model for ST32_baseline.csv already exists
Model for NST59_baseline.csv already exists
Model for ST20_baseline.csv already exists
Model for ST72_baseline.csv already exists
Model for NST29_baseline.csv already exists
Model for NST83_baseline.csv already exists
Model for ST50_baseline.csv already exists
Model for ST02_baseline.csv already exists
Model for NST69_baseline.csv already exists
Model for ST42_baseline.csv already exists
Model for ST26_baseline.csv already exists
Model for ST74_baseline.csv already exists
Model for ST34_baseline.csv already exists
Model for ST66_baseline.csv already exists
Model for NST47_baseline.csv already exists
Model for NST15_baseline.csv already exists
Model for NST55_baseline.csv already exists
Mo

---
## Constructing Observations

In [7]:
processed_physio_folder_path = "./Physiological Preprocessed/"

exp2_folder_path = processed_physio_folder_path + "Exp2"

exp2_takeover_times = pd.read_csv(
    "./AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)
exp2_takeover_times.iloc[:, 2:] = exp2_takeover_times.iloc[:, 2:].apply(pd.to_timedelta, unit="s")
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].apply(
    lambda x: x.split("T")[0] + "T" + x.split("T")[1].zfill(2)
)
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].astype(str)
exp2_takeover_times.drop(columns=["label_st"], inplace=True)
exp2_takeover_times.sort_values(by=["subject_id"], inplace=True)

for column in exp2_takeover_times.columns:
    if "TrigObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("TrigObs", "") + "TOR"}
        )
    elif "RepObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("RepObs", "Response")}
        )

exp2_takeover_times

,subject_id,DeerTOR,DetObsDeer,ResponseDeer,ConeTOR,DetObsCone,ResponseCone,FrogTOR,DetObsFrog,ResponseFrog,CanTOR,DetObsCan,ResponseCan,FA1TOR,DetObsFA1,ResponseFA1,FA2TOR,DetObsFA2,ResponseFA2
0,NST01,0 days 00:02:56.705100,0 days 00:02:59.093200,0 days 00:03:03.523800,0 days 00:06:56.214000,0 days 00:06:58.510900,0 days 00:06:58.690200,0 days 00:08:34.815700,0 days 00:08:38.845600,NaT,0 days 00:13:06.640800,0 days 00:13:12.859100,NaT,0 days 00:16:23.624000,NaT,NaT,0 days 00:18:02.245000,0 days 00:18:06.363000,NaT
2,NST03,0 days 00:13:35.204000,0 days 00:13:38.469300,0 days 00:13:39.824400,0 days 00:10:19.808800,0 days 00:10:21.816700,0 days 00:10:23.032200,0 days 00:04:19.471200,0 days 00:04:23.789000,NaT,0 days 00:17:07.400900,0 days 00:17:10.785900,NaT,0 days 00:06:18.340900,NaT,NaT,0 days 00:18:35.174900,0 days 00:18:42.818900,NaT
4,NST05,0 days 00:07:08.961300,0 days 00:07:10.742400,0 days 00:07:11.272600,0 days 00:12:23.166400,0 days 00:12:24.461700,0 days 00:12:38.273600,0 days 00:14:59.418600,0 days 00:15:00.724600,0 days 00:15:02.753600,0 days 00:04:28.071600,0 days 00:04:29.987500,0 days 00:04:31.402700,0 days 00:02:23.631400,0 days 00:02:24.742400,NaT,0 days 00:10:29.173600,0 days 00:10:30.421200,NaT
6,NST07,0 days 00:02:36.060300,0 days 00:02:39.121900,0 days 00:02:46.883100,0 days 00:06:11.856300,0 days 00:06:13.610700,0 days 00:06:15.090900,0 days 00:08:28.559300,0 days 00:08:31.772600,0 days 00:08:32.484500,0 days 00:12:00.085800,0 days 00:12:02.153100,0 days 00:12:02.817700,0 days 00:16:34.142400,0 days 00:16:37.435400,NaT,0 days 00:18:05.472400,0 days 00:18:08.195400,NaT
8,NST09,0 days 00:13:11.586070,0 days 00:13:13.970070,0 days 00:13:16.070470,0 days 00:08:18.826970,0 days 00:08:20.710570,0 days 00:08:22.473270,0 days 00:03:16.580170,0 days 00:03:21.515270,0 days 00:03:24.339770,0 days 00:15:36.305270,0 days 00:15:38.094270,0 days 00:15:39.460270,0 days 00:05:37.980670,0 days 00:05:52.104570,NaT,0 days 00:19:03.294270,0 days 00:19:07.322270,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,ST82,0 days 00:16:22.060600,0 days 00:16:25.019600,0 days 00:16:26.115600,0 days 00:11:27.337900,0 days 00:11:30.174000,0 days 00:11:31.206600,0 days 00:05:00.722700,0 days 00:05:05.274800,0 days 00:05:04.673200,0 days 00:02:09.414300,0 days 00:02:12.728300,NaT,0 days 00:08:01.118200,0 days 00:08:23.261900,NaT,0 days 00:13:10.705300,0 days 00:13:37.020600,NaT
83,ST84,0 days 00:13:08.533100,0 days 00:13:10.099000,0 days 00:13:15.842500,0 days 00:16:19.256299999,0 days 00:16:20.686300,0 days 00:16:22.944300,0 days 00:02:01.331000,0 days 00:02:02.753000,0 days 00:02:04.980400,0 days 00:08:02.602800,0 days 00:08:04.146900,0 days 00:05:04.680500,0 days 00:02:01.331000,0 days 00:02:02.753000,0 days 00:02:04.980400,0 days 00:05:00.889300,0 days 00:05:02.173200,0 days 00:05:04.680500
85,ST86,0 days 00:05:00.561600,0 days 00:05:02.025700,0 days 00:05:03.506499999,0 days 00:02:56.399600,0 days 00:02:57.203300,0 days 00:03:04.042600,0 days 00:13:40.438800,0 days 00:13:42.381800,0 days 00:13:45.033800,0 days 00:16:52.421300,0 days 00:16:54.161300,0 days 00:16:54.876300,0 days 00:08:11.486600,0 days 00:08:13.166100,NaT,0 days 00:10:30.702800,0 days 00:10:32.862900,NaT
87,ST88,0 days 00:17:33.644200,0 days 00:17:35.548200,0 days 00:17:38.642200,0 days 00:11:47.429500,0 days 00:11:49.730800,0 days 00:11:52.368200,0 days 00:05:03.127099999,0 days 00:05:07.203900,0 days 00:05:07.837400,0 days 00:02:22.286400,0 days 00:02:23.154300,NaT,0 days 00:08:03.736300,0 days 00:08:08.586700,NaT,0 days 00:13:32.521300,0 days 00:13:34.723000,NaT


In [8]:
def collect_observations(phsyiological_data_folder, include_latent_space, jump, takeover_times, tot):
    """
    Create the observations for the slow and fast takeover times.

    Args:
        phsyiological_data_dictionary (dict): A dictionary containing the segmented physiological data files.
        takeover_times (pd.DataFrame): A DataFrame containing the takeover times.
        driver_demographic_data (pd.DataFrame): A DataFrame containing the driver demographic data.
        window_length (int): The length of the window in minutes.
        window_step (int): The step size for the window

    Returns:
        list: A list of observations for the slow takeover times.
        list: A list of observations for the fast takeover times.
    """
    driving_observations_data = []
    takeover_observations_data = []

    for file in os.listdir(phsyiological_data_folder):
        # Split the file name into the participant and period
        f = file.split("_")
        participant = f[0]
        period = f[1].split(".")[0]

        if period != "driving":
            continue

        # print(f"Processing {participant} data")

        # Physiological data
        physio = pd.read_csv(phsyiological_data_folder + "/" + file, usecols=["Time", "ECG_Clean", "RSP_Clean", "EDA_Tonic", "EDA_Phasic"])
        physio["Time"] = pd.to_timedelta(physio["Time"])
        physio.set_index("Time", inplace=True)

        # Initialize the model
        model = ConvAutoencoder().to(device)
        criterion = nn.MSELoss()
        model.load_state_dict(torch.load(f"./Convolutional Autoencoder Models/Takeover Time/{participant}_model.pth"))
        model.eval()

        scalar = MinMaxScaler()
        data = scalar.fit_transform(physio)
        physio = pd.DataFrame(data, columns=physio.columns, index=physio.index)

        # Takeover Times
        participant_takeover_times = takeover_times[
            takeover_times["subject_id"] == participant
        ].copy()

        # convert every value to timedelta
        participant_takeover_times.iloc[:, 1:] = participant_takeover_times.iloc[:, 1:].apply(
            pd.to_timedelta, args=("s",), errors="coerce"
        )

        obstacles = ["Deer", "Cone", "Frog", "Can"]
        for obstacle in obstacles:

            obstacle_trigger_time = pd.to_timedelta(participant_takeover_times[f"{obstacle}TOR"].values[0], unit="s")
            minute_before_obstacle = obstacle_trigger_time - pd.Timedelta(seconds=60)

            if pd.isnull(obstacle_trigger_time):
                continue
            if pd.isnull(minute_before_obstacle):
                continue

            # Get the physiological data for the period
            obstacle_physio = physio.loc[obstacle_trigger_time - pd.Timedelta(seconds=4):obstacle_trigger_time + pd.Timedelta(seconds=8)]
            driving_physio = physio.loc[minute_before_obstacle- pd.Timedelta(seconds=4):obstacle_trigger_time + pd.Timedelta(seconds=8)]

            observations = []

            step_size = "0.001s"
            takeover_observations = []
            len_segment = pd.Timedelta(segment_size) / pd.Timedelta(step_size)
            while len(obstacle_physio) > 0:
                start_index = obstacle_physio.index[0]
                end_index = start_index + pd.Timedelta(segment_size)
                segmented_signal = obstacle_physio[:end_index]
                obstacle_physio = obstacle_physio[start_index + pd.Timedelta(jump) :]

                if len(segmented_signal) > len_segment:
                    length = len(segmented_signal) - len_segment
                    segmented_signal = segmented_signal.drop(segmented_signal.tail(int(length)).index)

                if len(segmented_signal) == len_segment:
                    segmented_signal = torch.tensor(segmented_signal.to_numpy(), dtype=torch.float32).to(device)
                    with torch.no_grad():
                        latent_space = model.encode(segmented_signal.permute(1, 0).unsqueeze(0))
                        reconstructed_signal = model.decode(latent_space).squeeze(0).permute(1, 0)
                    loss = criterion(reconstructed_signal, segmented_signal)
                    if include_latent_space:
                        pca_signal = PCA(n_components=1).fit_transform(latent_space.cpu().detach().numpy().squeeze()).squeeze()
                        segmented_observations = np.concatenate((pca_signal, [loss.item()]))
                    else:
                        segmented_observations.append(loss.item())

                    takeover_observations.append(segmented_observations)

            takeover_observations = np.stack(takeover_observations)

            driving_observations = []
            while len(driving_physio) > 0:
                start_index = driving_physio.index[0]
                end_index = start_index + pd.Timedelta(segment_size)
                segmented_signal = driving_physio[:end_index]
                driving_physio = driving_physio[start_index + pd.Timedelta(jump) :]

                if len(segmented_signal) > len_segment:
                    length = len(segmented_signal) - len_segment
                    segmented_signal = segmented_signal.drop(segmented_signal.tail(int(length)).index)

                if len(segmented_signal) == len_segment:
                    segmented_signal = torch.tensor(segmented_signal.to_numpy(), dtype=torch.float32).to(device)
                    with torch.no_grad():
                        latent_space = model.encode(segmented_signal.permute(1, 0).unsqueeze(0))
                        reconstructed_signal = model.decode(latent_space).squeeze(0).permute(1, 0)
                    loss = criterion(reconstructed_signal, segmented_signal)
                    if include_latent_space:
                        pca_signal = PCA(n_components=1).fit_transform(latent_space.cpu().detach().numpy().squeeze()).squeeze()
                        segmented_observations = np.concatenate((pca_signal, [loss.item()]))
                    else:
                        segmented_observations.append(loss.item())

                    driving_observations.append(segmented_observations)

            driving_observations = np.stack(driving_observations)

            driving_observations_data.append(driving_observations)
            takeover_observations_data.append(takeover_observations)

    return driving_observations_data, takeover_observations_data

In [9]:
def accuracy(
    driving_observations,
    takeover_observations,
    n_components_driving,
    n_components_takeover,
    n_mix_driving,
    n_mix_takeover,
    covariance_type,
    include_latent_space,
):
    iterations = 100

    accuracies = []
    true_positives_list = []
    false_positives_list = []
    true_negatives_list = []
    false_negatives_list = []

    for i in range(iterations):
        if i % 10 == 0:
            print(f"Iteration: {i}")

        # split the data
        driving_observations_train, driving_observations_test = train_test_split(
            driving_observations, test_size=0.3
        )
        takeover_observations_train, takeover_observations_test = train_test_split(
            takeover_observations, test_size=0.3
        )

        # concatenate the observations
        X_driving = None
        X_driving_lengths = []
        for data in driving_observations_train:
            if X_driving is None:
                X_driving = data
            else:
                X_driving = np.concatenate((X_driving, data))
            X_driving_lengths.append(len(data))

        X_takeover = None
        X_takeover_lengths = []
        for data in takeover_observations_train:
            if X_takeover is None:
                X_takeover = data
            else:
                X_takeover = np.concatenate((X_takeover, data))
            X_takeover_lengths.append(len(data))

        # initialize and fit the models
        driving_model = hmm.GMMHMM(
            n_components=n_components_driving, n_mix=n_mix_driving, covariance_type=covariance_type
        )
        takeover_model = hmm.GMMHMM(
            n_components=n_components_takeover,
            n_mix=n_mix_takeover,
            covariance_type=covariance_type,
        )

        # fit the models
        if not include_latent_space:
            X_slow = X_slow.reshape(-1, 1)
            X_fast = X_fast.reshape(-1, 1)
        driving_model.fit(X_driving, X_driving_lengths)
        takeover_model.fit(X_takeover, X_takeover_lengths)
        
        # score the models
        accuracy = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0

        for _, observation in enumerate(driving_observations_test):

            if driving_model.score(observation) > takeover_model.score(observation):
                accuracy += 1
                tn += 1
            else:
                fn += 1

        for _, observation in enumerate(takeover_observations_test):

            if takeover_model.score(observation) > driving_model.score(observation):
                accuracy += 1
                tp += 1
            else:
                fp += 1

        accuracy = accuracy / (len(driving_observations_test) + len(takeover_observations_test))
        accuracies.append(accuracy)

        true_positives_list.append(tp)
        false_positives_list.append(fp)
        true_negatives_list.append(tn)
        false_negatives_list.append(fn)

        if i % 10 == 0:
            print(f"Accuracy: {accuracy}")
            print(f"True Positives: {tp}")
            print(f"False Positives: {fp}")
            print(f"True Negatives: {tn}")
            print(f"False Negatives: {fn}")

    return (
        accuracies,
        true_positives_list,
        false_positives_list,
        true_negatives_list,
        false_negatives_list,
    )

In [10]:
tot = "3"
include_latent_space = [True, False]
jump_size = ["1s", "500ms", "100ms"]
n_components_slow = [1, 2, 3, 4]
n_mix_slow = [1, 2, 3, 4]
n_components_fast = [1, 2, 3, 4]
n_mix_fast = [1, 2, 3, 4]
covariance_types = ["full", "diag", "spherical"]

In [ ]:
for include_latent in include_latent_space:
    for jump in jump_size:
        print(f"Collecting observations every {jump} {'with latent space' if include_latent else ''}")
        driving_observations, takeover_observations = collect_observations(exp2_folder_path, include_latent, jump, exp2_takeover_times, tot)
        print(
            f"Collected {len(driving_observations)} driving observations and {len(takeover_observations)} takeover observations"
        )

        for n_slow in n_components_slow:
            for m_slow in n_mix_slow:
                for n_fast in n_components_fast:
                    for m_fast in n_mix_fast:
                        for cov in covariance_types:

                            # Check if the results already exist
                            if os.path.exists("results-to-ehmm.csv"):
                                results_df = pd.read_csv("results-to-ehmm.csv")
                                results = results_df[
                                        (results_df["TOT"] == int(tot)) 
                                        & (results_df["Latent Space"] == include_latent)
                                        & (results_df["Jump"] == jump)
                                        & (results_df["Components Slow"] == n_slow)
                                        & (results_df["Mixtures Slow"] == m_slow)
                                        & (results_df["Components Fast"] == n_fast)
                                        & (results_df["Mixtures Fast"] == m_fast)
                                        & (results_df["Covariance"] == cov)
                                    ]
                                if not results.empty:
                                    print("Results already exist for these parameters.")
                                    continue

                            print("-------------------------------------------------")
                            print(f"Slow Components: {n_slow}, Slow Mixtures: {m_slow}, Fast Components: {n_fast}, Fast Mixtures: {m_fast}, Covariance: {cov}")
                            try:
                                (
                                    accuracies,
                                    true_positives_list,
                                    false_positives_list,
                                    true_negatives_list,
                                    false_negatives_list,
                                ) = accuracy(
                                    driving_observations,
                                    takeover_observations,
                                    n_slow,
                                    n_fast,
                                    m_slow,
                                    m_fast,
                                    cov,
                                    include_latent,
                                )

                                # Accuracy
                                print(f"Average Accuracy: {np.mean(accuracies)}")
                                print(f"Standard Deviation: {np.std(accuracies)}")
                                print(f"Max Accuracy: {np.max(accuracies)}")
                                print(f"Min Accuracy: {np.min(accuracies)}")

                                # Find the index of the max accuracy
                                max_accuracy_index = accuracies.index(np.max(accuracies))
                                tp = true_positives_list[max_accuracy_index]
                                print(f"True Positives: {tp}")
                                fp = false_positives_list[max_accuracy_index]
                                print(f"False Positives: {fp}")
                                tn = true_negatives_list[max_accuracy_index]
                                print(f"True Negatives: {tn}")
                                fn = false_negatives_list[max_accuracy_index]
                                print(f"False Negatives: {fn}")

                                # check  if any of the values are zero
                                if tp + fp == 0 or tp + fn == 0:
                                    precision = 0
                                    recall = 0
                                    f1_score = 0
                                else:
                                    # Precision, Recall, and F1 Score
                                    precision = tp / (tp + fp)
                                    recall = tp / (tp + fn)
                                    f1_score = 2 * precision * recall / (precision + recall)
                                print(f'Precision: {precision}')
                                print(f'Recall: {recall}') 
                                print(f'F1 Score: {f1_score}')
                                print("-------------------------------------------------")
                                print()
                                results = [tot, include_latent, jump, n_slow, m_slow, n_fast, m_fast, cov, np.mean(accuracies), np.std(accuracies), np.max(accuracies), np.min(accuracies), tp, fp, tn, fn, precision, recall, f1_score]
                            except:
                                results = [tot, include_latent, jump, n_slow, m_slow, n_fast, m_fast, cov, None, None, None, None, None, None, None, None, None, None, None]

                            results_columns = ["TOT", "Latent Space", "Jump", "Components Slow", "Mixtures Slow", "Components Fast", "Mixtures Fast", "Covariance", "Mean Accuracy", "Standard Deviation", "Max Accuracy", "Min Accuracy", "True Positives", "False Positives", "True Negatives", "False Negatives", "Precision", "Recall", "F1 Score"]

                            if os.path.exists("results-to-ehmm.csv"):
                                results_df = pd.read_csv("results-to-ehmm.csv")
                                results_df = pd.concat([results_df, pd.DataFrame([results], columns=results_columns)])
                                results_df.to_csv("results-to-ehmm.csv", index=False)
                            else:
                                pd.DataFrame([results], columns=results_columns).to_csv("results-to-ehmm.csv", index=False)

Collected 320 driving observations and 320 takeover observations
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results already exist for these parameters.
Results alr

Model is not converging.  Current: -22269.987195211994 is not greater than -22269.986992876256. Delta is -0.00020233573741279542


Accuracy: 0.5052083333333334
True Positives: 4
False Positives: 92
True Negatives: 93
False Negatives: 3


Model is not converging.  Current: -24640.410875090547 is not greater than -24640.405197338787. Delta is -0.005677751760231331
Model is not converging.  Current: -20612.162939836453 is not greater than -20612.16264738046. Delta is -0.0002924559921666514
Model is not converging.  Current: -24967.427897354402 is not greater than -24967.425217885346. Delta is -0.0026794690566021018
Model is not converging.  Current: -23039.449179248157 is not greater than -23039.442877775105. Delta is -0.006301473051280482


Iteration: 10
Accuracy: 0.4895833333333333
True Positives: 1
False Positives: 95
True Negatives: 93
False Negatives: 3


...

Iteration: 90


Model is not converging.  Current: -21901.942788467233 is not greater than -21901.942123722096. Delta is -0.000664745137328282


Accuracy: 0.5
True Positives: 0
False Positives: 96
True Negatives: 96
False Negatives: 0


Model is not converging.  Current: -24526.16498691839 is not greater than -24526.153048224707. Delta is -0.011938693682168378
Model is not converging.  Current: -25151.2332764444 is not greater than -25151.231968175234. Delta is -0.0013082691657473333
Model is not converging.  Current: -22833.375283734666 is not greater than -22833.37471607154. Delta is -0.0005676631262758747
Model is not converging.  Current: -23108.598645083017 is not greater than -23108.59612314961. Delta is -0.002521933405660093
Model is not converging.  Current: -24630.65148791837 is not greater than -24630.649337938285. Delta is -0.0021499800859601237
Model is not converging.  Current: -24199.795954337547 is not greater than -24199.79264272174. Delta is -0.003311615808343049
Model is not converging.  Current: -25154.807107497607 is not greater than -25154.798979620075. Delta is -0.008127877532388084


Average Accuracy: 0.50046875
Standard Deviation: 0.0034622687197125622
Max Accuracy: 0.5104166666666666
Min Accuracy: 0.484375
True Positives: 2
False Positives: 94
True Negatives: 96
False Negatives: 0
Precision: 0.020833333333333332
Recall: 1.0
F1 Score: 0.04081632653061225
-------------------------------------------------

-------------------------------------------------
Slow Components: 4, Slow Mixtures: 3, Fast Components: 1, Fast Mixtures: 1, Covariance: diag
Iteration: 0
Accuracy: 0.5104166666666666
True Positives: 2
False Positives: 94
True Negatives: 96
False Negatives: 0
Iteration: 10
Accuracy: 0.5052083333333334
True Positives: 2
False Positives: 94
True Negatives: 95
False Negatives: 1
Iteration: 20
Accuracy: 0.5052083333333334
True Positives: 1
False Positives: 95
True Negatives: 96
False Negatives: 0
Iteration: 30
Accuracy: 0.5
True Positives: 5
False Positives: 91
True Negatives: 91
False Negatives: 5
Iteration: 40
Accuracy: 0.5
True Positives: 2
False Positives: 94
Tru